In [1]:
with open(r"C:\Users\umuts\Desktop\test.txt",encoding="utf8") as fh:
    train_lines = fh.readlines()

In [2]:
array_tutucu=[]
t_tutucu=[]
for asa in train_lines:
    k=asa.split()
    array_tutucu.append(k)
for a in array_tutucu:
    tu=tuple(a)
    t_tutucu.append(tu)

In [3]:
c_tutucu=[]
met_tutucu=[]
for x in t_tutucu:
    if len(x)==2:
        c_tutucu.append(x)
    elif len(x)<1:
        met_tutucu.append(c_tutucu)
        c_tutucu=[]

print(met_tutucu[0:5])
    

[[('Bunlar', 'O'), (',', 'O'), ('mücadelenin', 'O'), ('kesin', 'O'), ('galibini', 'O'), ('belirleyecek', 'O'), ('bir', 'O'), ('sayısal', 'O'), ('güce', 'O'), ('sahip', 'O'), ('bulunuyorlar', 'O')], [('Bugünkü', 'O'), ('mücadele', 'O'), (',', 'O'), ('Savaş', 'O'), ('sonrası', 'O'), ('Fedaral', 'B-LOCATION'), ('Almanya', 'I-LOCATION'), ('Cumhuriyeti', 'I-LOCATION'), ("'", 'O'), ('nin', 'O'), ('kuruluşundan', 'O'), ('bu', 'O'), ('yana', 'O'), ('14ncü', 'O'), (',', 'O'), ('iki', 'O'), ('Almanya', 'B-LOCATION'), ("'", 'O'), ('nın', 'O'), ('birlişmesinden', 'O'), ('sonra', 'O'), ('da', 'O'), ('3', 'O'), ('.', 'O'), ('ve', 'O'), ('20', 'O'), ('.', 'O'), ('yüzyılın', 'O'), ('son', 'O'), ('seçimi', 'O')], [('20', 'O'), ('.', 'O'), ('yüzyılın', 'O'), ('son', 'O'), ('genel', 'O'), ('seçimlerinde', 'O'), ('de', 'O'), ('daha', 'O'), ('önce', 'O'), ('olduğu', 'O'), ('gibi', 'O'), ('merkez', 'O'), ('sağın', 'O'), ('lokomotifi', 'O'), ("'", 'O'), ('Rekortmen', 'O'), ('Başbakan', 'O'), ("'", 'O'), ('He

In [4]:
from itertools import chain

import nltk
import sklearn
import scipy.stats

import sklearn_crfsuite
from sklearn_crfsuite import scorers,CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics

In [5]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        
       
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

In [6]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [ ]:
#Creating the train and test set
X = [sent2features(s) for s in met_tutucu]
y = [sent2labels(s) for s in met_tutucu]

In [8]:
from sklearn.model_selection import cross_val_predict, cross_val_score

#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False
          )


In [9]:

pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

C:\Users\umuts\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


                precision    recall  f1-score   support

    B-LOCATION       0.88      0.75      0.81      1126
B-ORGANIZATION       0.83      0.77      0.79       873
      B-PERSON       0.82      0.71      0.76      1603
    I-LOCATION       0.74      0.46      0.57       211
I-ORGANIZATION       0.76      0.81      0.78       864
      I-PERSON       0.82      0.90      0.85       803
             O       0.98      0.99      0.99     44115

      accuracy                           0.97     49595
     macro avg       0.83      0.77      0.79     49595
  weighted avg       0.96      0.97      0.96     49595



In [10]:
#Tuning the parameters manually, setting c1 = 10
crf2 = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [11]:
pred = cross_val_predict(estimator=crf2, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                precision    recall  f1-score   support

    B-LOCATION       0.70      0.54      0.61      1126
B-ORGANIZATION       0.61      0.46      0.53       873
      B-PERSON       0.62      0.50      0.55      1603
    I-LOCATION       0.15      0.05      0.08       211
I-ORGANIZATION       0.47      0.57      0.52       864
      I-PERSON       0.59      0.72      0.65       803
             O       0.97      0.98      0.98     44115

      accuracy                           0.93     49595
     macro avg       0.59      0.55      0.56     49595
  weighted avg       0.93      0.93      0.93     49595



In [12]:
with open(r"C:\Users\umuts\Desktop\train.txt",encoding="utf8") as fh:
    train_lines = fh.readlines()

In [13]:
array_tutucu=[]
t_tutucu=[]
for asa in train_lines:
    k=asa.split()
    array_tutucu.append(k)
for a in array_tutucu:
    tu=tuple(a)
    t_tutucu.append(tu)

In [14]:
c_tutucu=[]
met_tutucu=[]
for x in t_tutucu:
    if len(x)==2:
        c_tutucu.append(x)
    elif len(x)<1:
        met_tutucu.append(c_tutucu)
        c_tutucu=[]

print(met_tutucu[0:5])
    

[[('ATİLLA', 'B-PERSON'), ('Mutman', 'I-PERSON'), (',', 'O'), ('İzmir', 'B-LOCATION'), ('milletvekili', 'O')], [('Gecenin', 'O'), ('erken', 'O'), ('saatlerinde', 'O'), ('de', 'O'), (',', 'O'), ('belli', 'O'), ('olacak', 'O'), ('Avrupa', 'B-LOCATION'), ("'", 'O'), ('nın', 'O'), ('en', 'O'), ('büyük', 'O'), ('ülkesinin', 'O'), ('hükümetini', 'O'), ('kimin', 'O'), ('yöneteceği', 'O')], [('Bilgisayarlı', 'O'), ('telefon', 'O'), ('klübesi', 'O')], [('Kozmopolit', 'O'), ('bir', 'O'), ('ulus', 'O')], [('Yıllık', 'O'), ('10', 'O'), ('milyon', 'O'), ('ton', 'O'), ('kapasitesi', 'O'), ('bulunan', 'O'), ('limana', 'O'), ('150', 'O'), ('bin', 'O'), ('DWT', 'O'), ('tonluk', 'O'), ('gemiler', 'O'), ('yanaşabilecek', 'O')]]


In [24]:
#Creating the train and test set
X = [sent2features(s) for s in met_tutucu]
y = [sent2labels(s) for s in met_tutucu]

In [25]:
from sklearn.model_selection import cross_val_predict, cross_val_score

#Creating the CRF model
crf3 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False
          )


In [26]:

pred = cross_val_predict(estimator=crf3, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                precision    recall  f1-score   support

    B-LOCATION       0.95      0.91      0.93      8821
B-ORGANIZATION       0.94      0.90      0.92      8316
      B-PERSON       0.94      0.92      0.93     13290
    I-LOCATION       0.85      0.77      0.81      1284
I-ORGANIZATION       0.89      0.88      0.88      5860
      I-PERSON       0.91      0.94      0.92      6314
             O       0.99      1.00      0.99    376111

      accuracy                           0.99    419996
     macro avg       0.92      0.90      0.91    419996
  weighted avg       0.99      0.99      0.99    419996



In [27]:
#Tuning the parameters manually, setting c1 = 10
crf4 = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [28]:
pred = cross_val_predict(estimator=crf4, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                precision    recall  f1-score   support

    B-LOCATION       0.89      0.70      0.78      8821
B-ORGANIZATION       0.84      0.75      0.79      8316
      B-PERSON       0.79      0.77      0.78     13290
    I-LOCATION       0.59      0.36      0.44      1284
I-ORGANIZATION       0.70      0.71      0.70      5860
      I-PERSON       0.74      0.86      0.80      6314
             O       0.98      0.99      0.99    376111

      accuracy                           0.96    419996
     macro avg       0.79      0.73      0.76    419996
  weighted avg       0.96      0.96      0.96    419996



In [29]:
with open(r"C:\Users\umuts\Desktop\dev.txt",encoding="utf8") as fh:
    train_lines = fh.readlines()

In [30]:
array_tutucu=[]
t_tutucu=[]
for asa in train_lines:
    k=asa.split()
    array_tutucu.append(k)
for a in array_tutucu:
    tu=tuple(a)
    t_tutucu.append(tu)

In [31]:
c_tutucu=[]
met_tutucu=[]
for x in t_tutucu:
    if len(x)==2:
        c_tutucu.append(x)
    elif len(x)<1:
        met_tutucu.append(c_tutucu)
        c_tutucu=[]

print(met_tutucu[0:5])
    

[[('Lav', 'O'), ('mermisinin', 'O'), ('sokak', 'O'), ('lambası', 'O'), ('direğine', 'O'), ('ve', 'O'), ('tabelaya', 'O'), ('çarptıktan', 'O'), ('sonra', 'O'), ('pencere', 'O'), ('pervazını', 'O'), ('parçalayarak', 'O'), ('birinci', 'O'), ('kat', 'O'), ('tavanına', 'O'), ('saplandığı', 'O'), ('belirlenirken', 'O'), (',', 'O'), ('Aksaray', 'B-ORGANIZATION'), ('Polis', 'I-ORGANIZATION'), ('Karakolu', 'I-ORGANIZATION'), ('olarak', 'O'), ('kullanılan', 'O'), ('birinci', 'O'), ('katta', 'O'), ('büyük', 'O'), ('çaplı', 'O'), ('hasar', 'O'), ('meydana', 'O'), ('geldi', 'O')], [('Başkan', 'O'), ('Yılmaz', 'B-PERSON'), ("'", 'O'), ('ın', 'O'), ('sürpriz', 'O'), ('kararla', 'O'), (',', 'O'), ('eski', 'O'), ('yöneticilerle', 'O'), ('bugün', 'O'), ('bir', 'O'), ('toplantı', 'O'), ('yapacağını', 'O'), ('açıklaması', 'O'), (',', 'O'), ('dikkatlerin', 'O'), ('Trabzonspor', 'B-ORGANIZATION'), ("'", 'O'), ('a', 'O'), ('çevrilmesine', 'O'), ('ve', 'O'), ('Fenerbahçe', 'B-ORGANIZATION'), ('maçı', 'O'), ('

In [32]:
#Creating the train and test set
X = [sent2features(s) for s in met_tutucu]
y = [sent2labels(s) for s in met_tutucu]

In [33]:
from sklearn.model_selection import cross_val_predict, cross_val_score

#Creating the CRF model
crf5 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False
          )


In [34]:

pred = cross_val_predict(estimator=crf5, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                precision    recall  f1-score   support

    B-LOCATION       0.90      0.76      0.83       942
B-ORGANIZATION       0.84      0.76      0.80       842
      B-PERSON       0.83      0.79      0.81      1400
    I-LOCATION       0.78      0.44      0.56       107
I-ORGANIZATION       0.73      0.71      0.72       589
      I-PERSON       0.78      0.87      0.82       680
             O       0.98      0.99      0.99     40972

      accuracy                           0.97     45532
     macro avg       0.83      0.76      0.79     45532
  weighted avg       0.97      0.97      0.97     45532



In [35]:
#Tuning the parameters manually, setting c1 = 10
crf6 = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [36]:
pred = cross_val_predict(estimator=crf6, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                precision    recall  f1-score   support

    B-LOCATION       0.67      0.49      0.57       942
B-ORGANIZATION       0.62      0.47      0.54       842
      B-PERSON       0.58      0.49      0.53      1400
    I-LOCATION       0.31      0.04      0.07       107
I-ORGANIZATION       0.40      0.46      0.42       589
      I-PERSON       0.54      0.73      0.62       680
             O       0.97      0.98      0.98     40972

      accuracy                           0.93     45532
     macro avg       0.58      0.52      0.53     45532
  weighted avg       0.93      0.93      0.93     45532

